# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pandas scikit-learn catboost
!pip install scikit-optimize

import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_validate, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, VotingRegressor, StackingRegressor
from sklearn.neural_network import MLPRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from xgboost import XGBRegressor
from google.colab import files
from scipy.stats import uniform, randint
from catboost import CatBoostRegressor
import numpy as np
from sklearn.linear_model import Ridge
from google.colab import files
from scipy.stats import uniform, randint
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

train = pd.read_csv("/content/drive/MyDrive/dacon_1/candidates_04_01.csv")

#validation set 분리

In [ ]:
X = train.drop(columns=["ID", "Inhibition"])
y = train["Inhibition"]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.1, random_state=42)

#Bayes Search

In [ ]:
from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, f1_score
import numpy as np

pbar = None

def custom_score(y_true, y_pred):
    from sklearn.metrics import mean_squared_error
    from scipy.stats import pearsonr

    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    y_range = y_true.max() - y_true.min()
    norm_rmse = rmse / y_range if y_range != 0 else 0

    if len(np.unique(y_true)) > 1 and len(np.unique(y_pred)) > 1:
        pearson_corr = pearsonr(y_true, y_pred)[0]
    else:
        pearson_corr = 0
    score = 0.5 * (1 - min(norm_rmse, 1)) + 0.5 * pearson_corr


    return score

scoring = make_scorer(custom_score, greater_is_better=True)

cv = KFold(n_splits=5, shuffle=True, random_state=42)

rf = RandomForestRegressor(random_state=42)

param_space = {
     'bootstrap': [True],
     'max_depth': (30,45), #37
     'max_features': (0.3, 0.6), #0.4528
     'max_samples': (0.5, 0.8), #0.6591
     'min_samples_leaf': (3, 7), #5
     'min_samples_split': (15, 30), #24
     'min_weight_fraction_leaf': (0.005, 0.012), #0.009611784839406745
     'n_estimators': (1500, 2000)} #1693


bayes_search = BayesSearchCV(
    estimator=rf,
    search_spaces=param_space,
    n_iter=30,
    cv=cv,
    scoring=scoring,
    n_jobs=-1,
    verbose=2,
    random_state=42,
    error_score='raise'
)

bayes_search.fit(X_train, y_train)

print("Best rf Parameters (Bayesian):", bayes_search.best_params_)
print("Best Mean CV Score:", bayes_search.best_score_)

best_rf = bayes_search.best_estimator_
best_rf.fit(X_train, y_train)

print("--- rf (bayes Tuned) on Validation ---")
y_pred = best_rf.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
y_range = y_val.max() - y_val.min()
norm_rmse = rmse / y_range
pearson_corr, _ = pearsonr(y_val, y_pred)
score = 0.5 * (1 - min(norm_rmse, 1)) + 0.5 * pearson_corr
print(f'Normalized RMSE: {norm_rmse:.12f}')
print(f'Pearson Correlation: {pearson_corr:.12f}')
print(f'Score: {score:.12f}')

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

In [ ]:
import joblib

joblib.dump(best_rf, "./RandomForest.pkl")

['./CatBoost.pkl']